In [1]:
# %pip install langchain
# %pip install typing
# %pip install openai
# !pip install openai==0.28
# !pip install langchain typing openai statsmodels seaborn matplotlib
# !pip install langchain_community
# !pip install langchain-openai

In [2]:
import sys
import os
import warnings
from dotenv import load_dotenv

import pandas as pd

from langchain.llms import OpenAI
from pydantic import BaseModel, Field

In [3]:
load_dotenv()
warnings.simplefilter(action='ignore', category=UserWarning)
sys.path.append(os.path.abspath(".."))

In [4]:
from utils.filter_utils import(
    filter_for_participant_counts, 
    filter_for_regularity, 
    filter_for_goal_str_length,
    print_participant_general_states,
    align_goal_with_day_they_were_done
)

from utils.llm_utils import (
    embed_with_retries
)

In [ ]:
run1_path = "../data/raw/daily-goals/run1_dailyGoals.csv"
run2_path = "../data/raw/daily-goals/run2_dailyGoals.csv"

In [7]:
cols = ["ParticipantIdentifier", "ResultIdentifier", "Answers", "trial_date"]
run1_raw = pd.read_csv(run1_path)[cols]
run2_raw = pd.read_csv(run2_path)[cols]

In [8]:
run1_wide = (
    run1_raw.pivot_table(
        index=["ParticipantIdentifier", "trial_date"],  # rows
        columns="ResultIdentifier",                     # new columns
        values="Answers",                               # cell values
        aggfunc="first"                                 # in case of duplicates
    )
    .reset_index()
)

run2_wide = (
    run2_raw.pivot_table(
        index=["ParticipantIdentifier", "trial_date"],  # rows
        columns="ResultIdentifier",                     # new columns
        values="Answers",                               # cell values
        aggfunc="first"                                 # in case of duplicates
    )
    .reset_index()
)

In [9]:
wide_goal_DF = pd.concat([run1_wide, run2_wide], axis=0).reset_index(drop=True)

In [10]:
def basic_counts(df):
    # 1. Number of participants
    n_participants = df['ParticipantIdentifier'].nunique()

    # 2. Entries per participant
    counts = df['ParticipantIdentifier'].value_counts()

    min_entries = counts.min()
    max_entries = counts.max()

    print("Number of participants:", n_participants)
    print("Min entries per participant:", min_entries)
    print("Max entries per participant:", max_entries)

In [11]:
basic_counts(run1_wide)

Number of participants: 90
Min entries per participant: 1
Max entries per participant: 83


In [12]:
basic_counts(run2_wide)

Number of participants: 93
Min entries per participant: 5
Max entries per participant: 85


In [13]:
basic_counts(wide_goal_DF)

Number of participants: 183
Min entries per participant: 1
Max entries per participant: 85


In [14]:
selfReport_cols = [
    "DAILY_goal1_confidence",
    "DAILY_goal1_consequences",
    "DAILY_goal1_effort",
    "DAILY_goal1_importance",
    "DAILY_goal1_report",
    "DAILY_goal1_motivationExternal",
    "DAILY_goal1_motivationInternal",

    "DAILY_goal2_confidence",
    "DAILY_goal2_consequences",
    "DAILY_goal2_effort",
    "DAILY_goal2_importance",
	"DAILY_goal2_report",
    "DAILY_goal2_interaction_eachOther",
    "DAILY_goal2_motivationExternal",
    "DAILY_goal2_motivationInternal"
]


run1_self_report_path = "/Users/farhan/projects/NROC90/data/raw/self_report/run1_survey_results.csv"
run2_self_report_path = "/Users/farhan/projects/NROC90/data/raw/self_report/run2_selfReport.csv"

In [15]:
run1_self_report = pd.read_csv(run1_self_report_path)[cols]
run1_self_report = run1_self_report.loc[run1_self_report["ResultIdentifier"].isin(selfReport_cols)]

run1_self_report = (
    run1_self_report.pivot_table(
        index=["ParticipantIdentifier", "trial_date"],  # rows
        columns="ResultIdentifier",                     # new columns
        values="Answers",                               # cell values
        aggfunc="first"                                 # in case of duplicates
    ).dropna().reset_index()
)

In [16]:
run2_self_report = pd.read_csv(run2_self_report_path)
run2_self_report.columns = run2_self_report.columns.str.replace(r'^sr_', '', regex=True)
run2_self_report = run2_self_report[["ParticipantIdentifier", "trial_date"] + selfReport_cols]

In [17]:
wide_self_report_DF = pd.concat([run1_self_report, run2_self_report], axis=0).dropna().reset_index(drop=True)

In [18]:
basic_counts(run1_self_report)
basic_counts(run2_self_report)
basic_counts(wide_self_report_DF)

Number of participants: 89
Min entries per participant: 2
Max entries per participant: 82
Number of participants: 93
Min entries per participant: 85
Max entries per participant: 85
Number of participants: 182
Min entries per participant: 2
Max entries per participant: 84


___

In [19]:
basic_counts(wide_goal_DF)
print("-----")
basic_counts(wide_self_report_DF)

Number of participants: 183
Min entries per participant: 1
Max entries per participant: 85
-----
Number of participants: 182
Min entries per participant: 2
Max entries per participant: 84


In [20]:
goal_report_merged = pd.merge(
    wide_goal_DF,
    wide_self_report_DF,
    on=["ParticipantIdentifier", "trial_date"],
    how="inner"
).dropna().reset_index(drop=True)

In [21]:
goal_report_merged.head()

,ParticipantIdentifier,trial_date,DAILY_goal1_set,DAILY_goal2_set,DAILY_goal1_confidence,DAILY_goal1_consequences,DAILY_goal1_effort,DAILY_goal1_importance,DAILY_goal1_motivationExternal,DAILY_goal1_motivationInternal,DAILY_goal1_report,DAILY_goal2_confidence,DAILY_goal2_consequences,DAILY_goal2_effort,DAILY_goal2_importance,DAILY_goal2_interaction_eachOther,DAILY_goal2_motivationExternal,DAILY_goal2_motivationInternal,DAILY_goal2_report
0,0501ba67-3406-4779-aff1-878a0e9f7885,2022-10-27,exercise,deconstruct paper,4,2,3,6,3,5,0,2,5,6,7,-2,6,5,0
1,0501ba67-3406-4779-aff1-878a0e9f7885,2022-10-28,write ted script,exercise,3,3,7,5,4,5,0,5,2,4,7,-2,5,7,100
2,0501ba67-3406-4779-aff1-878a0e9f7885,2022-10-29,script,quizzical creation,4,4,6,7,5,7,0,7,4,6,7,-2,6,6,62
3,0501ba67-3406-4779-aff1-878a0e9f7885,2022-10-30,script,study psy and hisa,5,5,6,7,7,3,11,4,4,7,7,-2,2,5,100
4,0501ba67-3406-4779-aff1-878a0e9f7885,2022-10-31,study psy and hisa,preview d26 and review c19,4,4,6,7,2,5,76,3,2,7,7,-4,2,7,0


In [22]:
basic_counts(goal_report_merged)

Number of participants: 182
Min entries per participant: 2
Max entries per participant: 84


In [23]:
goal_report_merged['trial_date'] = pd.to_datetime(goal_report_merged['trial_date'])
df = goal_report_merged.sort_values(['ParticipantIdentifier', 'trial_date'])
df['day_diff'] = df.groupby('ParticipantIdentifier')['trial_date'].diff().dt.days

In [24]:
df = align_goal_with_day_they_were_done(df)
df = filter_for_goal_str_length(df, 1)
df = df.loc[df['day_diff'] == 1]
df = filter_for_participant_counts(df, 40)

Total rows: 9055
Unique participants: 179
Average entries per participant: 50.59
Total rows: 9055
Unique participants: 179
Average entries per participant: 50.59
Total rows: 7788
Unique participants: 112
Average entries per participant: 69.54


In [25]:
df.to_csv("../data/proc/goal_selfReport_merged_filtered_wide.csv", index=False)

In [27]:
df[[
    "ParticipantIdentifier", 
    "trial_date",
    "DAILY_goal1_set", 
    "DAILY_goal2_set"
    ]
].to_csv("../data/proc/goal_filtered_wide.csv", index=False)

In [29]:
goal_long = df.melt(
    id_vars=["ParticipantIdentifier", "trial_date"],
    value_vars=["DAILY_goal1_set", "DAILY_goal2_set"],
    var_name="ResultIdentifier",   # or "ResponseIdentifier" if you prefer
    value_name="Answers"
).to_csv("../data/proc/goal_filtered_long.csv", index=False)